# EDA216 - Lecture 2

During the lectures and exercises we'll try out SQL-code in Jupyter
Notebooks, and to do that we begin by evaluating:

In [ ]:
%load_ext sql

This time I'll use a database defined for a Stanford course:

In [ ]:
%sql sqlite:///db02.db

Everything in the code sections is evaluated
as [Python](https://www.python.org/) code (more on that later), but
lines beginning with `%sql` will be seen as an SQL statement, and all
lines following `%%sql` will also be evaluated as SQL statements.

## Joins

The following example is taken almost verbatim
from [Jennifer Widom](https://www.youtube.com/watch?v=5Myx_X2sWOI)
(with some name alterations to comply with the coding conventions I've
been using).

We have the following schema:

~~~~ {.sql}
CREATE TABLE colleges(c_name text, state text, enrollment int);
CREATE TABLE students(s_id int, s_name text, gpa real, size_hs int);
CREATE TABLE applications(s_id int, c_name text, major text, decision text);
~~~~


A typical selection:

In [ ]:
%%sql
SELECT DISTINCT s_name, major
FROM   students s, applications a
WHERE  s.s_id = a.s_id

The comma in the `FROM` clause gives us a cross join:

In [ ]:
%%sql
SELECT DISTINCT s_name, major
FROM   students s CROSS JOIN applications a
WHERE  s.s_id = a.s_id

This can be rewritten as a *inner join* on the `s_id`:

In [ ]:
%%sql
SELECT DISTINCT s_name, major
FROM   students s INNER JOIN applications a
ON     s.s_id = a.s_id

and as `INNER JOIN` is the default join operator, we can omit the
`INNER`:

In [ ]:
%%sql
SELECT DISTINCT s_name, major
FROM   students s JOIN applications a
ON     s.s_id = a.s_id

We can add conditions to our inner joins, we start with another
typical cross join:

In [ ]:
%%sql
SELECT s_name, gpa
FROM   students s, applications a
WHERE  s.s_id = a.s_id AND
       size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

This can be rewritten to a `INNER JOIN` on the id's, and the rest of
the condition can be a `WHERE` clause:

In [ ]:
%%sql
SELECT s_name, gpa
FROM   students s JOIN applications a
ON     s.s_id = a.s_id
WHERE  size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

but we can also put all conditions in the `ON` clause:

In [ ]:
%%sql
SELECT s_name, gpa
FROM   students s JOIN applications a
ON     s.s_id = a.s_id AND
       size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

It will make little difference, the query engine is supposed to
optimize our queries either way.

Now let's try with three tables -- we want to see some application
information:

In [ ]:
%%sql
SELECT a.s_id, s_name, gpa, a.c_name, enrollment
FROM   applications a, students s, colleges c
WHERE  a.s_id = s.s_id AND
       a.c_name = c.c_name

We can again rewrite this as joins:

In [ ]:
%%sql
SELECT a.s_id, s_name, gpa, a.c_name, enrollment
FROM   applications a JOIN students s JOIN colleges c
ON     a.s_id = s.s_id AND
       a.c_name = c.c_name

## Other kinds of joins

Back to our original query:

In [ ]:
%%sql
SELECT DISTINCT s_name, major
FROM   students s INNER JOIN applications a
ON     s.s_id = a.s_id

Since our two tables have the attribute `s_id` in common, we can use a
*natural join*:

In [ ]:
%%sql
SELECT DISTINCT s_name, major
FROM   students NATURAL JOIN applications

The natural join will eliminate the duplicate attributes:

In [ ]:
%%sql
SELECT *
FROM   students NATURAL JOIN applications

Observe that there's only one `s_id` in the output.

We can use natural joins with more complicated queries:

In [ ]:
%%sql
SELECT s_name, gpa
FROM   students s JOIN applications a
ON     s.s_id = a.s_id
WHERE  size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

Once again we have a common attribute, so we can write:

In [ ]:
%%sql
SELECT s_name, gpa
FROM   students s NATURAL JOIN applications a
WHERE  size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

Using natural join is a bit risky, since we might have other
attributes with the same name (it can easily happen over time, if we
modify our tables) -- a better way of expressing this is to use a
`JOIN` with `USING`:

In [ ]:
%%sql
SELECT s_name, gpa
FROM   students s JOIN applications a USING(s_id)
WHERE  size_hs < 1000 AND
       major = 'CS' AND
       c_name = 'Stanford'

Here we explicitly tell what attributes we want to equate.

A slightly bigger example -- we want to find all pairs with the same
Grade Point Average (GPA):

In [ ]:
%%sql
SELECT s1.s_id, s1.s_name, s1.gpa, s2.s_id, s2.s_name, s2.gpa
FROM   students s1, students s2
WHERE  s1.gpa = s2.gpa AND
       s1.s_id < s2.s_id

In [ ]:
%%sql
SELECT s1.s_id, s1.s_name, s1.gpa, s2.s_id, s2.s_name, s2.gpa
FROM   students s1 JOIN students s2 USING(gpa)
WHERE  s1.s_id < s2.s_id

We will return to joins when we get into relational algebra later in
the course.